In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
file = "tpcds.csv"

# read a csv file where the columnas are delimitted by # and load this to a dataframe
# // Generated by WCA for GP
import pandas as pd

# Load the CSV file into a Pandas data frame
df = pd.read_csv(file, delimiter='#')

In [3]:
df.head()

,tables,joins,predicates,db2,actual
0,"web_sales,date_dim,catalog_sales,date_dim,cust...","cd_demo_sk=c.c_current_cdemo_sk,c.c_customer_s...","d_year,=,2002",126.928,265.964
1,"web_sales,date_dim,catalog_sales,date_dim,cust...","cd_demo_sk=c.c_current_cdemo_sk,c.c_customer_s...","d_year,=,2002",129.740,287.028
2,"web_sales,date_dim,catalog_sales,date_dim,cust...","cd_demo_sk=c.c_current_cdemo_sk,c.c_customer_s...","d_year,=,2002",126.928,258.644
3,"web_sales,date_dim,catalog_sales,date_dim,cust...","cd_demo_sk=c.c_current_cdemo_sk,c.c_customer_s...","d_year,=,2002",129.420,269.756
4,"web_sales,date_dim,catalog_sales,date_dim,cust...","cd_demo_sk=c.c_current_cdemo_sk,c.c_customer_s...","d_year,=,2001",129.008,261.952


In [4]:
df.shape

(13654, 5)

# Labeling queries based on memory usage level

In [5]:
num_templates = 20

In [6]:
df.shape


(13654, 5)

In [7]:
len(np.unique(df['actual']))

6349

In [8]:
# Generated by WCA for GP
df['template'] = pd.qcut(df['actual'], q=num_templates, labels=False, duplicates='drop')

In [9]:
df.shape

(13654, 6)

In [10]:
# generate code to train test split a dataframe

# import train_test_split

from sklearn.model_selection import train_test_split


# Generated by WCA for GP

# Split the dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Save the train and test sets as CSV files
# train_df.to_csv('train_tpcds.csv', index=False, sep='#')
test_df.to_csv('test_tpcds.csv', index=False, sep='#')

In [11]:
train_df['template'].value_counts()

template
1     1223
4      877
0      677
5      656
8      606
11     569
3      562
15     558
17     546
13     546
7      536
16     534
18     533
14     532
10     531
12     521
9      502
2      273
6      141
Name: count, dtype: int64

In [12]:
median_size = int(np.median(train_df['template'].value_counts()))

In [13]:
train_df.shape

(10923, 6)

In [14]:
train_df['template'].unique()

array([10,  7, 16,  1,  0,  2,  4, 11, 14, 12, 15,  8, 18, 13,  3,  9, 17,
        6,  5])

In [15]:
# how to create an empty dataframe that has the same columns as an existing dataframe?
# Generated by WCA for GP
# Create an empty DataFrame with the same columns as df
new_df = pd.DataFrame(columns=train_df.columns)

for i in train_df['template'].unique():
    #print('template: {}'.format(i))
    length = train_df[train_df['template'] == i].shape[0]
    #print('length: {}'.format(length))

    if length == median_size:
        new_df = pd.concat([new_df, train_df[train_df['template']==i].copy()], axis=0)
    elif length > median_size:
        new_df = pd.concat([new_df, train_df[train_df['template']==i].sample(median_size)])
    elif length < median_size:
        ratio = math.ceil(median_size / length)
        temp_df = train_df[train_df['template']==i].copy().sample(n=length * ratio, replace=True)
        temp_df = temp_df.sample(median_size)
        new_df = pd.concat([new_df, temp_df])


/var/folders/6f/_thrtvnj5dj0h94ccbgkdb9r0000gn/T/ipykernel_21634/173364556.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, temp_df])


In [16]:
new_df.shape

(10374, 6)

In [17]:
new_df['template'].value_counts()

template
10    546
15    546
6     546
17    546
9     546
3     546
13    546
18    546
8     546
12    546
7     546
14    546
11    546
4     546
2     546
0     546
1     546
16    546
5     546
Name: count, dtype: int64

In [18]:
train_df = new_df.reset_index(drop=True)

In [19]:
train_df.head()

,tables,joins,predicates,db2,actual,template
0,"web_sales,date_dim,catalog_sales,date_dim,cust...","cd_demo_sk=c.c_current_cdemo_sk,c.c_customer_s...","d_year,=,2002",129.008,250.268,10
1,"store_sales,date_dim,customer_address,item,cat...","ss_ie_sk=i_item_sk,ss_sold_date_sk=d_date_sk,s...","d_year,=,2002,d_moy,=,5,ca_gmt_offset,=,-5",127.548,259.508,10
2,"store_sales,date_dim,store,household_demographics",store_sales.ss_sold_date_sk=date_dim.d_date_sk...,"d_dom,between,1 3,hd_buy_potential,=>,10000,hd...",72.656,246.404,10
3,"store_sales,date_dim,store,household_demographics",store_sales.ss_sold_date_sk=date_dim.d_date_sk...,"d_dom,between,1 3,hd_buy_potential,=,501-1000,...",72.656,246.072,10
4,"store_sales,date_dim,store,household_demographics",store_sales.ss_sold_date_sk=date_dim.d_date_sk...,"d_dom,between,1 3,hd_buy_potential,=>,10000,hd...",72.640,246.436,10


In [20]:
train_df.dtypes

tables         object
joins          object
predicates     object
db2           float64
actual        float64
template       object
dtype: object

In [36]:
train_df['template'] = train_df['template'].astype('category')

In [22]:
len(train_df['template'].unique())

19

In [ ]:
train_df.to_csv('train_tpcds.csv', index=False, sep='#')